# Introducing Generic PDE Support in CUQI

Here we build a simple Bayesian problem in which the forward model is a differential equation model, 1D Poisson problem in particular.

## Learning objectives of this notebook:
- Build a differential equation forward model in CUQIpy.
- Solve the differential equation-constrained Bayesian problem using the defined model.
- Explore the effect of changing noise and observations.


## The forward model

The PDE model we consider here is a Poisson problem with heterogeneous coefficients:

$$ \nabla \cdot (\kappa(x) \nabla u(x)) = f(x) \in \Omega,$$

where $\kappa$ is the diffusivity coefficient, $u(x)$ is the PDE solution (potential), and $f(x)$ is the source term. This equation, for example, can model a steady-state heat conductivity problem where $\kappa(x)$ is the thermal diffusivity and $u(x)$ is the temperature. $f(x)$ can be a heat source or sink term.

We assume the domain $\Omega$ is the 1D interval $[0,L]$, where $L$ is the domain length. We can think of this as heat conductivity problem in a 1D rod and we assume a zero Dirichlet boundary conditions on the boundaries. In the 1D case, the PDE system reduces to an ODE of the form
$$ \frac{d}{dx} \left(\kappa(x) \frac{d}{dx} u(x)\right) = f(x) \in [0,L],$$

$$u(0)=u(L)=0$$





## The Bayesian parameters

We assume that we know that the source term $f(x)$ is a Gaussian pulse with unknown location $x_0$ and unknown magnitude $a$:

$$f(x) = a e^{ - 50\left(\frac{(x - x_0)}{L}\right)^2} $$

And we are after inferring $\theta = (a,x_0)$ (2 scalar parameters) given the measurement of the temperature $u$ everywhere in the domain (or alternatively in part of the domain).

The data $d$ is then given by:

$$ d = \mathcal{G}(\theta) + \eta$$


where $\eta$ is the measurement noise and $\mathcal{G}$ is the forward model operator which maps the source term to the observations. We assume that the prior distribution of $\theta$ is uniform and the noise distribution is Gaussian. 


## The discretization 

We discretize the differential operator $ \frac{d}{dx} \left(\kappa(x) \frac{d}{dx} . \right)$ using the finite difference method (forward difference discretization in particular). The result discretized system is of the form 

$$\mathbf{A}\mathbf{u} = \mathbf{f}(\theta)$$

where $\mathbf{A}$, $\mathbf{u}$, and $\mathbf{f}$ are the differential operator, discretized temperature, and the discretized source term respectively.

## The code

To solve this problem in CUQIpy we need to perform the following steps:
- Create the forward problem:
    - Create the PDE form
    - Create a cuqi.pde.SteadyStateLinearPDE object
    - Create a cuqi.model.PDEModel object
- Create and solve the inverse problem:
    - Create the likelihood
    - Create synthetic data
    - Create the prior
    - Create the posterior
    - Sample the posterior

We start by importing the libraries we need

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
import cuqi

We set the finite difference grid parameters and construct the source term (the gaussian pulse):

In [ ]:
dim = 50 #Number of nodes
L = 10 # Length of the domain
dx = L/(dim-1) # grid spacing

grid_sol = np.linspace(dx, L, dim-1, endpoint=False)
grid_obs = grid_sol

source = lambda a, x0: a*np.exp( -50 * ( (grid_sol - x0)/L)**2 )

Then we create the discretized differential operator using `cuqi.operator.FirstOrderFiniteDifference`. We choose the diffusivity $\kappa$ to be a step function.

In [ ]:
FOFD_operator = cuqi.operator.FirstOrderFiniteDifference(dim-1,bc_type='zero').get_matrix().todense()

kappa = np.ones(dim)
kappa[np.where(grid_sol>L/2)] = 2

diff_operator = FOFD_operator.T @np.diag(kappa) @ FOFD_operator

We create the PDE form which consists of the differential operator and the right hand side, and is a function of the Bayesian parameter x. 

In [ ]:
poisson_form = lambda x: (diff_operator, dx**2*source(x[0],x[1]))

We create the CUQI PDEModel, in this case a `SteadyStateLinearPDE` model.

In [ ]:
CUQI_pde = cuqi.pde.SteadyStateLinearPDE(poisson_form, grid_sol=grid_sol, grid_obs=grid_obs)

Lets take a look at the object CUQI_pde

In [ ]:
CUQI_pde

In [ ]:
help(CUQI_pde)

The model `CUQI_pde` has three main methods: 

1. assemble, which assembles the differential operator and the RHS given the Bayesian parameter x.
2. solve, which solves the PDE.
3. observe, which applies an observation operators on the PDE solution (e.g. extracting final temperature at specific or random points).

In the following we assemble, solve and apply the observation operator for this Poisson problem for an exact solution `x_exact`.

In [ ]:
x_exact = np.array([10,3])
CUQI_pde.assemble(x_exact)
sol = CUQI_pde.solve()
observed_sol = CUQI_pde.observe(sol)

And we plot the solution, observed solution and the source term:

In [ ]:
plt.plot(grid_sol,source(x_exact[0], x_exact[1]), label = 'source term')
plt.plot(grid_sol, sol, color='k', label= 'solution')
plt.plot(grid_obs, observed_sol, linestyle='--', color='orange' ,label='observed solution')
plt.legend()

We build the CUQI forward model including creating the domain and range geometries

In [ ]:
domain_geometry = cuqi.distribution.Discrete(variables=['magnitude','location'])
range_geometry = cuqi.distribution.Continuous1D(grid_obs)

model = cuqi.model.PDEModel(CUQI_pde,range_geometry,domain_geometry)

We can look at the model:

In [ ]:
model

We then create the likelihood and the synthetic data:

In [ ]:
SNR = 100
sigma = np.linalg.norm(observed_sol)/SNR

likelihood = cuqi.distribution.GaussianCov(model, sigma**2*np.eye(range_geometry.dim))
data = likelihood(x=x_exact).sample()

We can now look at the data, solution, observed solution and the source term:

In [ ]:
plt.plot(grid_sol,source(x_exact[0], x_exact[1]), label = 'source term')
plt.plot(grid_sol, sol, color='k', label= 'solution')
plt.plot(grid_obs, observed_sol, linestyle='--', color='orange' ,label='observed solution')
plt.plot(grid_obs, data, linestyle='-', color='g' ,label='observed solution')
plt.legend()

We the create the prior, and the posterior:

In [ ]:
prior = cuqi.distribution.Uniform(np.array([1,0]), np.array([15,L]))
posterior = cuqi.distribution.Posterior(likelihood,prior,data)

And sample the posterior:

In [ ]:
np.random.seed(0)
mySampler = cuqi.sampler.MetropolisHastings(posterior)
samples = mySampler.sample_adapt(5000)
samples.plot_ci(95,exact=x_exact)

Let's look at the chains:

In [ ]:
samples.plot_chain([0,1])

We can apply `burnthin` to remove the burn-in and plot the confidence interval again:

In [ ]:
new_samples = samples.burnthin(500)
new_samples.plot_ci(95,exact=x_exact)

* We can try a case where SNR is 10, for example `SNR = 10`.

* We can also try observing only in the right half of the domain by setting `grid_obs = grid_sol[int(dim/2.):]` (keeping `SNR=100`)